In [2]:
pip install gurobipy

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB 660.6 kB/s eta 0:00:16
    --------------------------------------- 0.2/10.2 MB 2.5 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.2 MB 3.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.2 MB 4.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.2 MB 5.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/10.2 MB 5.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/10.2 MB 6.0 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.2 MB 6.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/10.2 MB 6.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.8/10.2 MB 6.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.0/10.2 MB 6.5 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.2 MB 6.5 MB/s eta 0:00:02
   -

In [3]:
import numpy as np
import pandas as pd

In [12]:
nut_req= pd.read_csv(r"C:\Users\16479\Downloads\nutrient_requirements.csv")
nut_con= pd.read_csv(r"C:\Users\16479\Downloads\nutrient_content.csv")
food_pref= pd.read_csv(r"C:\Users\16479\Downloads\food_preferences.csv")
food_cat= pd.read_csv(r"C:\Users\16479\Downloads\food_categories.csv")

In [6]:
import gurobipy as gp
from gurobipy import GRB

In [13]:
model = gp.Model("OptiDiet")

In [14]:
food_vars = model.addVars(food_cat.Food_Item, name="foods", lb=0, vtype=GRB.CONTINUOUS)

In [19]:
food_vars

{'Food_1': <gurobi.Var *Awaiting Model Update*>,
 'Food_2': <gurobi.Var *Awaiting Model Update*>,
 'Food_3': <gurobi.Var *Awaiting Model Update*>,
 'Food_4': <gurobi.Var *Awaiting Model Update*>,
 'Food_5': <gurobi.Var *Awaiting Model Update*>,
 'Food_6': <gurobi.Var *Awaiting Model Update*>,
 'Food_7': <gurobi.Var *Awaiting Model Update*>,
 'Food_8': <gurobi.Var *Awaiting Model Update*>,
 'Food_9': <gurobi.Var *Awaiting Model Update*>,
 'Food_10': <gurobi.Var *Awaiting Model Update*>,
 'Food_11': <gurobi.Var *Awaiting Model Update*>,
 'Food_12': <gurobi.Var *Awaiting Model Update*>,
 'Food_13': <gurobi.Var *Awaiting Model Update*>,
 'Food_14': <gurobi.Var *Awaiting Model Update*>,
 'Food_15': <gurobi.Var *Awaiting Model Update*>,
 'Food_16': <gurobi.Var *Awaiting Model Update*>,
 'Food_17': <gurobi.Var *Awaiting Model Update*>,
 'Food_18': <gurobi.Var *Awaiting Model Update*>,
 'Food_19': <gurobi.Var *Awaiting Model Update*>,
 'Food_20': <gurobi.Var *Awaiting Model Update*>,
 'Food_21

In [20]:
# Nutrition Constraints
for _, nutrient in nut_req.iterrows():
    min_intake = nutrient['Min_Requirement']
    max_intake = nutrient['Max_Requirement']
    nutrient_name = nutrient['Nutrient']

   
    intake = gp.quicksum(nut_con.loc[nut_con['Food_Item'] == food, nutrient_name].item() * food_vars[food] 
                         for food in food_cat['Food_Item'])
    
    model.addRange(intake, min_intake, max_intake, f"Nutrient_balance_{nutrient_name}")

In [23]:
# Dietary Preferences Constraints
total_dietary_needs = {
    'Vegetarian': food_pref['Veggie_grams'].iloc[0],
    'Vegan': food_pref['Vegan_grams'].iloc[0],
    'Kosher': food_pref['Kosher_grams'].iloc[0],
    'Halal': food_pref['Halal_grams'].iloc[0],
    'All': food_pref['All_grams'].iloc[0]
}

# Ensure the total grams for Vegetarian, Vegan, Kosher, and Halal are satisfied
for diet_type, total_grams in total_dietary_needs.items():
    if diet_type != 'All': 
        food_items = food_cat[food_cat['Is_' + diet_type] == 1]['Food_Item']
        model.addConstr(
            gp.quicksum(food_vars[food] for food in food_items) >= total_grams,
            f"min_{diet_type}_grams"
        )

# Ensure the total grams of all food items meet the 'All' dietary need
model.addConstr(
    gp.quicksum(food_vars[food] for food in food_cat['Food_Item']) == total_dietary_needs['All'],
    "total_all_grams"
)

<gurobi.Constr *Awaiting Model Update*>

In [24]:
for food in food_cat.Food_Item:
    model.addConstr(food_vars[food] <= 0.03 * gp.quicksum(food_vars[food] for food in food_cat.Food_Item), "variety_%s" % food)

In [25]:
model.setObjective(gp.quicksum(food_vars[food] * food_cat.loc[food_cat['Food_Item'] == food, 'Cost_per_gram'].item() for food in food_cat.Food_Item), GRB.MINIMIZE)

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 429 rows, 180 columns and 50916 nonzeros
Model fingerprint: 0x4bf393f6
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [5e-02, 2e-01]
  Bounds range     [4e+03, 8e+03]
  RHS range        [8e+03, 6e+05]
Presolve removed 246 rows and 0 columns
Presolve time: 0.02s
Presolved: 183 rows, 180 columns, 21864 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9256393e+04   2.082249e+05   0.000000e+00      0s

Solved in 189 iterations and 0.04 seconds (0.02 work units)
Infeasible model


In [26]:
print(f"The number of constraints in the model (excluding non-negativity): {model.NumConstrs}")

The number of constraints in the model (excluding non-negativity): 429
